In [1]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import pandas as pd
import numpy as np
from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'keys.json'

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)



SAMPLE_SPREADSHEET_ID = '1Ycg7zTxds9DZnDvTrFcyNNKuTUxg6Yy6WF0a8Wc02WQ'



service = build('sheets', 'v4', credentials=creds)


sheet = service.spreadsheets()


# Dimensions

In [2]:
rows_m = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                  range="managers!a1:c14").execute()
managers = rows_m.get('values')
print(managers)

[['manager_id', 'd_manager', 'd_club'], ['1e9c5562-0cfc-11ea-abe1-c412f533dba1', 'manager #8', 'club #2'], ['43756fa4-57a0-11e9-ab9a-c412f533dba1', 'manager #11', 'club #4'], ['543888c4-fbb3-11e9-abdc-c412f533dba1', 'manager #12', 'club #3'], ['5dd88f32-6501-11ea-abf3-c412f533dba1', 'manager #10', 'club #2'], ['9a87c608-52dc-11ea-abeb-c412f533dba1', 'manager #9', 'club #3'], ['abed5496-ca88-11e9-abb8-c412f533dba1', 'manager #5', 'club #4'], ['ad52c7a8-a752-11e7-8115-c412f533dba1', 'manager #7', 'club #1'], ['b55a233c-5df6-11ea-abf2-c412f533dba1', 'manager #2', 'club #4'], ['b832aac3-fcda-11ea-ac18-c412f533dba1', 'manager #13', 'club #3'], ['d12fe765-b444-11e9-abb3-c412f533dba1', 'manager #1', 'club #1'], ['db06b8bc-bfb2-11e7-8116-c412f533dba1', 'manager #4', 'club #2'], ['e463b03f-2d74-11eb-ac1f-c412f533dba1', 'manager #3', 'club #1'], ['f44596c6-12cd-11eb-ac1c-c412f533dba1', 'manager #6', 'club #2']]


In [3]:
df_man = pd.DataFrame(managers[1:], columns=managers[0])

In [4]:
rows_source = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                range="leads!d1:e3338").execute()
source = rows_source.get('values')
print(source)

[['d_utm_source', 'l_manager_id'], ['viber', '1e9c5562-0cfc-11ea-abe1-c412f533dba1'], ['instagram', 'd12fe765-b444-11e9-abb3-c412f533dba1'], ['instagram', '43756fa4-57a0-11e9-ab9a-c412f533dba1'], ['', '00000000-0000-0000-0000-000000000000'], ['vk', '5dd88f32-6501-11ea-abf3-c412f533dba1'], ['sms', 'b55a233c-5df6-11ea-abf2-c412f533dba1'], ['', 'db06b8bc-bfb2-11e7-8116-c412f533dba1'], ['instagram', '43756fa4-57a0-11e9-ab9a-c412f533dba1'], ['google', '5dd88f32-6501-11ea-abf3-c412f533dba1'], ['', '9a87c608-52dc-11ea-abeb-c412f533dba1'], ['instagram', 'db06b8bc-bfb2-11e7-8116-c412f533dba1'], ['sms', 'b55a233c-5df6-11ea-abf2-c412f533dba1'], ['viber', '1e9c5562-0cfc-11ea-abe1-c412f533dba1'], ['instagram', 'db06b8bc-bfb2-11e7-8116-c412f533dba1'], ['instagram', 'db06b8bc-bfb2-11e7-8116-c412f533dba1'], ['', 'b832aac3-fcda-11ea-ac18-c412f533dba1'], ['instagram', '9a87c608-52dc-11ea-abeb-c412f533dba1'], ['instagram', 'ad52c7a8-a752-11e7-8115-c412f533dba1'], ['', '43756fa4-57a0-11e9-ab9a-c412f533dba

In [5]:
df_source = pd.DataFrame(source[1:], columns=source[0])

In [6]:
df_source['manager_id'] = df_source['l_manager_id']

In [7]:
result = pd.merge(df_source, df_man, on="manager_id")

In [8]:
result_metrics = result.drop(labels = ['l_manager_id', 'manager_id'], axis=1)

In [9]:
result_metrics

,d_utm_source,d_manager,d_club
0,viber,manager #8,club #2
1,viber,manager #8,club #2
2,viber,manager #8,club #2
3,insta,manager #8,club #2
4,,manager #8,club #2
...,...,...,...
2766,utm_source,manager #5,club #4
2767,,manager #5,club #4
2768,,manager #5,club #4
2769,yandex,manager #5,club #4


# Metrics

# Количество заявок

In [10]:
rows_leads = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                  range="leads!a1:f3338").execute()
leads = rows_leads.get('values')


In [11]:
df_leads = pd.DataFrame(leads[1:], columns=leads[0])

In [12]:
lead_counts= len(df_leads)

# Количество мусорных заявок

In [13]:
df_garbage = df_leads.loc[df_leads['l_client_id'] == '00000000-0000-0000-0000-000000000000'] 

In [14]:
garbage_counts = len(df_garbage)

# Количество новых заявок (не было заявок и покупок от этого клиента раньше)

In [15]:
rows_transactions = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                  range="transactions!a1:d29001").execute()
transactions = rows_transactions.get('values')


In [16]:
df_transactions = pd.DataFrame(transactions[1:], columns=transactions[0])

In [17]:
df_transactionss = df_transactions.drop_duplicates(subset = ['transaction_id', 'l_client_id', ], keep = False)
df_leadss = df_leads.drop_duplicates(subset = 'lead_id', keep = False)

In [18]:
df_result3 = pd.merge(df_leadss, df_transactionss, on = 'l_client_id', how = 'left')

In [19]:
df_result3 = df_result3.drop_duplicates(subset = ['lead_id', 'l_client_id'], keep = False)

In [20]:
new = len(df_result3)

# Количество покупателей (кто купил в течение недели после заявки)


In [21]:
rows_transactions = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                  range="transactions!a1:d29001").execute()
transactions = rows_transactions.get('values')

In [22]:
rows_leads= sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                  range="leads!a1:f3338").execute()
leads = rows_leads.get('values')

In [23]:
df_transactions = pd.DataFrame(transactions[1:], columns=transactions[0])
df_leads = pd.DataFrame(leads[1:], columns=leads[0])

In [24]:
df_leads['created_at_start'] = df_leads['created_at']
df_transactions['created_at_finish'] = df_transactions['created_at']

In [25]:
df_transactions = df_transactions.loc[(df_transactions['m_real_amount'] != '0')]
df_leads = df_leads.loc[df_leads['l_client_id'] != '00000000-0000-0000-0000-000000000000'] 

In [26]:
df_cust = pd.merge(df_transactions, df_leads, how = 'inner', on = 'l_client_id')

In [27]:
df_time_start= pd.to_datetime(df_cust['created_at_start'])
df_time_finish = pd.to_datetime(df_cust['created_at_finish'])

In [28]:
time = df_time_finish - df_time_start
df_cust['time'] = time

In [29]:
df_cust2 = abs(df_cust['time'].dt.days)

In [30]:
df_cust['time_days'] = df_cust2

In [31]:
df_cust_res = df_cust.loc[df_cust['time_days'] < 7]

In [32]:
customers = len(df_cust_res)

# Количество новых покупателей (кто купил в течение недели после заявки, и не покупал раньше)


In [33]:
df_cust_res2 = df_cust_res.drop_duplicates(subset = 'transaction_id' , keep = False)

In [34]:
customers2 = len(df_cust_res2)

# Доход от покупок новых покупателей


In [35]:
df_cust_res3 = pd.to_numeric(df_cust_res2['m_real_amount'])

In [36]:
total = df_cust_res3.sum()

# Metrics

In [37]:
result2 = pd.DataFrame(data = [lead_counts, garbage_counts, new, customers, customers2, total], index = ['заявки', 'мусорные заявки', 'новые заявки', 'покупатели', 'новые покупатели', 'доход'], dtype=np.int)
result2

,0
заявки,3337
мусорные заявки,824
новые заявки,3097
покупатели,807
новые покупатели,568
доход,14714783


In [38]:
result3 = result2.pivot_table(columns = ['заявки', 'мусорные заявки', 'новые заявки', 'покупатели', 'новые покупатели', 'доход'])

# to google sheets


In [39]:
from __future__ import print_function
import os.path
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
import pandas as pd

from google.oauth2 import service_account

SCOPES = ['https://www.googleapis.com/auth/spreadsheets']
SERVICE_ACCOUNT_FILE = 'keys.json'

creds = None
creds = service_account.Credentials.from_service_account_file(
        SERVICE_ACCOUNT_FILE, scopes=SCOPES)


SAMPLE_SPREADSHEET_ID = '1j-abixA4GBXLqjJZ32HM9rjotK3omdFNlrD27vVTkXM'



service = build('sheets', 'v4', credentials=creds)

sheet = service.spreadsheets()


In [40]:
response = service.spreadsheets().values().append(spreadsheetId=SAMPLE_SPREADSHEET_ID, valueInputOption = 'RAW', range= 'df2!a1:a5', 
                                                 body = dict(
                                                 values = result3.T.reset_index().T.values.tolist()
                                                 )).execute()

In [41]:
response = service.spreadsheets().values().append(spreadsheetId=SAMPLE_SPREADSHEET_ID, valueInputOption = 'RAW', range= 'df1!a1:c2770', 
                                                 body = dict(
                                                 values = result_metrics.T.reset_index().T.values.tolist()
                                                 )).execute()